In [1]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
import types

### Scrapped Data for Point Tables

In [2]:
url_list = ["1","2","3","4","8","26","49"]
url_pts = "https://www.prokabaddi.com/sifeeds/kabaddi/live/json/{}_standing.json"


In [3]:
def parse(dic, c_df):
    if type(dic) is dict:
        for key,value in dic.items():
            if key == "team":
                if isinstance(dic[key],(list,tuple)):
                    df1 = pd.DataFrame()
                    for i in range(0,len(dic[key])):
                        item = value[i]
                        di = {}
                        for k,va in item.items():
                            if k == "match_result":
                                break;
                            if k in di:
                                print("yes")
                               # di[k].append(va)
                            else:
                                di[k] = va

                        tb = json_normalize(di)
                        if len(c_df) >0:
                            c_df = pd.concat([c_df, tb])
                        else:
                            c_df = tb

                    return c_df
                else:
                    table = json_normalize(dic[key])
            else:
                c_df = parse(dic[key],c_df)

    elif isinstance(dic,(tuple,list)):
        for i in range(0,len(dic)):
            item = dic[i]
            c_df = parse(item,c_df)
    else:
        print("not x:",dic)
    return c_df


In [4]:
df = pd.DataFrame()
for i in range(0, len(url_list)):
    jsonData = requests.get(url_pts.format(url_list[i])).json()
    jsons = jsonData
    table = jsonData["standings"]
    df2 = pd.DataFrame()
    df2 = parse(table, df2)
    df2["season"] =i+1
    if len(df) > 0:
        df = pd.concat([df,df2])
    else:
        df = df2
                   

not x: 1
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 0
not x: 
not x: 2
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 0
not x: 
not x: 3
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 0
not x: 
not x: 4
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 0
not x: 
not x: 8
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 3
not x: Zone A
not x: 4
not x: Zone B
not x: 26
not x: 1
not x: 0
not x: Jul 19, 2019
not x: 3
not x: Zone A
not x: 4
not x: Zone B
not x: 49
not x: 1
not x: 0
not x: Sep 29, 2019
not x: 0
not x: 


In [5]:
df.to_csv("point_table.csv",sep='\t')

### Scrapping Team Data

In [17]:
url_list = ["1_1","1_2","1_3","1_4","1_8","1_26","1_49"]

team_won = [3,5,6,6,6,1,0]

feature_list= ["96","133","140","13","97","98","15","95","99","134","20","135","136","137"]

feature_name = ["total_points_scored", "total_points_conceded","avg_points_scored","successful_raids","raid_points","avg_raid_points","successful_tackles",
               "tackle_points", "avg_tackle_points","super_raid","super_tackles","do_or_die_raid_points","all_outs_inflicted","all_outs_conceded" ]
url = "https://www.prokabaddi.com/sifeeds/kabaddi/static/json/{}_{}_stats.json"
prev_tab = pd.DataFrame()



In [19]:
for i in range(0,len(url_list)):

    #jsonData = requests.get(url.format(url_list[i])).json()
    prev_col = pd.DataFrame()
    for j in range(0, len(feature_list)):
        print(i,j)
        url_s = url.format(url_list[i],feature_list[j])
        jsonData = requests.get(url_s).json()
        table = json_normalize(jsonData['data'])
        table.rename(columns = {'value':feature_name[j]}, inplace = True)
        table.rename(columns = {'rank':feature_name[j]+"_rank"}, inplace = True)
        table.rename(columns = {'match_played':feature_name[j]+"_match_played"}, inplace = True)
        if len(prev_col) > 0:
            prev_col = pd.merge(prev_col,table,on=["team_id","team_name"])
        else:
            prev_col = table

    prev_col["season"] = i+1
    
    team = team_won[i]
    
    prev_col['winner'] = prev_col["team_id"].apply(lambda x: 1 if x == team else 0)
    
    if len(prev_tab) >0 :
        prev_tab = pd.concat([prev_col,prev_tab])
    else:
        prev_tab = prev_col



0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13


In [21]:
file_name = "team.csv"
prev_tab.to_csv(file_name, sep='\t')


### Merge Point table Data and Team Data

In [22]:
import pandas as pd


df_team = pd.read_csv("team.csv", sep='\t')
df_point = pd.read_csv("point_table.csv", sep='\t')

print(df_team.shape)
print(df_point.shape)

print(df_team.head())

(68, 47)
(68, 27)
   Unnamed: 0  total_points_scored_match_played  total_points_scored_rank  \
0           0                                19                         1   
1           1                                19                         2   
2           2                                19                         3   
3           3                                20                         4   
4           4                                19                         5   

   team_id          team_name  total_points_scored  \
0        2  Dabang Delhi K.C.                  709   
1        4    Bengal Warriors                  676   
2        1    Bengaluru Bulls                  656   
3        7      Puneri Paltan                  652   
4       28   Haryana Steelers                  650   

   total_points_conceded_match_played  total_points_conceded_rank  \
0                                  19                           7   
1                                  19                   

In [23]:
df_m = pd.merge(df_team,df_point, on=['team_id','team_name','season'])

In [24]:
df_m.drop(columns="Unnamed: 0_x", axis=1, inplace=True)

In [25]:
df_m.dropna(axis=1, how="all", inplace=True)

In [26]:
df_m.drop(columns="Unnamed: 0_y", axis=1, inplace=True)

In [27]:
df_m.to_csv("team_data.csv",sep=";")